В этой практической тетрадке мы изучим различные методы нормализации текста – стемматизацию и лемматизацию – и сравним, как работают основные библиотеки обработки естественного языка для русского и английского языков.

Мы рассмотрим:

* Разницу между стемматизацией и лемматизацией
* Сравнение работы библиотек NLTK, spaCy, PyMorphy2 и Natasha
* Создание комплексных пайплайнов препроцессинга для разных языков
* Практические упражнения для закрепления материала

In [2]:
# Установка необходимых библиотек
!pip install nltk spacy pymorphy2 natasha
!python -m spacy download en_core_web_sm
!python -m spacy download ru_core_news_sm

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

import spacy
import pymorphy2
from natasha import Segmenter, MorphVocab, NewsEmbedding, NewsMorphTagger, Doc

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 91.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 3.2 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=85a03849213bef7608e8d359a5df6dad22349a5eb0c377eb4dde5d98486ebb53
  Stored in directory: /root/.cache/pip/wheels/1a/bf/a1/4cee4f7678c68c5875ca89eaccf460593539805c3906722228
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26098 sha256=50fc3e874f84879c2b152620edc53953bf2834d5b86f8d418f40064e531cb1b6
  Stored in directory: /root/.cache/pip/wheels/65/c3/c3/238bf93c243597857edd94ddb0577faa74a8e16e9585896e83
Successfully built docopt intervaltree
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


**1. Сравнение стемматизации и лемматизации**

In [10]:
# Тестовые тексты
russian_text = """
В России проживают народы с разными традициями и верованиями.
Русские люди всегда с интересом относились к культуре соседних народов.
Петр Первый ввел многие европейские обычаи.
Екатерина Великая переписывалась с французскими просветителями.
Современные россияне любят путешествовать по разным странам мира.
"""

english_text = """
The United States is home to people with diverse traditions and beliefs.
Americans have always been interested in the cultures of neighboring countries.
George Washington established many governmental customs.
Thomas Jefferson corresponded with French intellectuals.
Modern Americans enjoy traveling to different countries around the world.
"""

1.1 Стемматизация с помощью NLTK. Стемматизация - это процесс нахождения основы слова путем отбрасывания аффиксов (окончаний, суффиксов). Стемматизаторы используют набор правил без словарей.




In [ ]:
from nltk.stem import SnowballStemmer
import re

# Создаем стемматизаторы для английского и русского языков
english_stemmer = SnowballStemmer("english")
russian_stemmer = SnowballStemmer("russian")

# Функция для токенизации и стемматизации
def stem_text(text, stemmer):
    # Приводим к нижнему регистру и токенизируем
    words = re.findall(r'\b\w+\b', text.lower())
    # Применяем стемматизацию
    stemmed_words = [stemmer.stem(word) for word in words]
    return stemmed_words

# Стемматизация русского текста
russian_stemmed = stem_text(russian_text, russian_stemmer)
print("Стемматизированные русские слова:")
for i, word in enumerate(russian_stemmed[:15]):  # Выводим первые 15 слов
    print(f"{i+1}. {word}")

# Стемматизация английского текста
english_stemmed = stem_text(english_text, english_stemmer)
print("\nСтемматизированные английские слова:")
for i, word in enumerate(english_stemmed[:15]):  # Выводим первые 15 слов
    print(f"{i+1}. {word}")

Стемматизированные русские слова:
1. машин
2. обучен
3. станов
4. все
5. бол
6. популярн
7. в
8. современ
9. мир
10. компан
11. использ
12. алгоритм
13. машин
14. обучен
15. для

Стемматизированные английские слова:
1. natur
2. languag
3. process
4. help
5. comput
6. understand
7. and
8. generat
9. human
10. languag
11. research
12. are
13. develop
14. better
15. model


1.2 Лемматизация с помощью разных библиотек. Лемматизация - это процесс приведения слова к его словарной (нормальной) форме. Лемматизаторы используют словари и учитывают грамматические особенности.

1.2.1 NLTK (только для английского)

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

In [ ]:
# Инициализация лемматизатора
lemmatizer = WordNetLemmatizer()

# Функция для конвертации POS-тегов NLTK в формат WordNet
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # По умолчанию существительное

# Функция для лемматизации с учетом части речи
def lemmatize_with_pos(text):
    # Токенизация и определение части речи
    tokens = word_tokenize(text.lower())
    tagged = pos_tag(tokens)

    # Лемматизация с учетом части речи
    lemmas = []
    for word, tag in tagged:
        if word.isalpha():  # Исключаем числа и знаки препинания
            wordnet_pos = get_wordnet_pos(tag)
            lemmas.append(lemmatizer.lemmatize(word, wordnet_pos))

    return lemmas

# Лемматизация английского текста
english_lemmas_nltk = lemmatize_with_pos(english_text)
print("Лемматизированные английские слова (NLTK):")
for i, word in enumerate(english_lemmas_nltk[:15]):
    print(f"{i+1}. {word}")

# Примечание: NLTK не имеет встроенного лемматизатора для русского языка
print("\nNLTK не поддерживает лемматизацию русского языка напрямую.")

Лемматизированные английские слова (NLTK):
1. natural
2. language
3. processing
4. help
5. computer
6. understand
7. and
8. generate
9. human
10. language
11. researcher
12. be
13. develop
14. good
15. model

NLTK не поддерживает лемматизацию русского языка напрямую.


1.2.2 spaCy (для обоих языков)

In [ ]:
# Загрузка моделей spaCy
nlp_en = spacy.load("en_core_web_sm")
nlp_ru = spacy.load("ru_core_news_sm")

# Функция для лемматизации с помощью spaCy
def lemmatize_with_spacy(text, nlp):
    doc = nlp(text)
    lemmas = [token.lemma_ for token in doc if token.is_alpha]
    return lemmas

# Лемматизация английского текста
english_lemmas_spacy = lemmatize_with_spacy(english_text, nlp_en)
print("Лемматизированные английские слова (spaCy):")
for i, word in enumerate(english_lemmas_spacy[:15]):
    print(f"{i+1}. {word}")

# Лемматизация русского текста
russian_lemmas_spacy = lemmatize_with_spacy(russian_text, nlp_ru)
print("\nЛемматизированные русские слова (spaCy):")
for i, word in enumerate(russian_lemmas_spacy[:15]):
    print(f"{i+1}. {word}")

Лемматизированные английские слова (spaCy):
1. natural
2. language
3. processing
4. help
5. computer
6. understand
7. and
8. generate
9. human
10. language
11. researcher
12. be
13. develop
14. well
15. model

Лемматизированные русские слова (spaCy):
1. машинный
2. обучение
3. становиться
4. всё
5. более
6. популярный
7. в
8. современный
9. мир
10. компания
11. использовать
12. алгоритм
13. машинный
14. обучение
15. для


1.2.3 PyMorphy2 (для русского языка)

In [ ]:
# Инициализация анализатора
morph = pymorphy2.MorphAnalyzer()

# Функция для лемматизации с помощью PyMorphy2
def lemmatize_with_pymorphy(text):
    # Токенизация
    words = re.findall(r'\b\w+\b', text.lower())

    # Лемматизация
    lemmas = [morph.parse(word)[0].normal_form for word in words]
    return lemmas

# Лемматизация русского текста
russian_lemmas_pymorphy = lemmatize_with_pymorphy(russian_text)
print("Лемматизированные русские слова (PyMorphy2):")
for i, word in enumerate(russian_lemmas_pymorphy[:15]):
    print(f"{i+1}. {word}")

1.2.4 Natasha (для русского языка)

In [ ]:
# Инициализация компонентов Natasha
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

# Функция для лемматизации с помощью Natasha
def lemmatize_with_natasha(text):
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)

    lemmas = []
    for token in doc.tokens:
        if token.text.isalpha():  # Исключаем числа и знаки препинания
            token.lemmatize(morph_vocab)
            lemmas.append(token.lemma)

    return lemmas

# Лемматизация русского текста
russian_lemmas_natasha = lemmatize_with_natasha(russian_text)
print("Лемматизированные русские слова (Natasha):")
for i, word in enumerate(russian_lemmas_natasha[:15]):
    print(f"{i+1}. {word}")

1.3 Сравнительная таблица результатов

Для наглядности сравним результаты обработки одних и тех же слов разными методами.

In [15]:
import pandas as pd

# Выбираем несколько интересных слов из русского текста для сравнения
russian_words = ["России", "проживают", "народы", "традициями", "верованиями",
                "русские", "людей", "относились", "культуре", "Петр", "ввел", "обычаи"]

# Создаем DataFrame для сравнения
comparison_ru = []
for word in russian_words:
    stem = russian_stemmer.stem(word.lower())
    lemma_spacy = None

    # Находим лемму в spaCy
    doc = nlp_ru(word)
    for token in doc:
        lemma_spacy = token.lemma_

    lemma_pymorphy = morph.parse(word.lower())[0].normal_form

    # Находим лемму в Natasha
    doc = Doc(word)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    for token in doc.tokens:
        token.lemmatize(morph_vocab)
        lemma_natasha = token.lemma

    comparison_ru.append({
        "Слово": word,
        "Стем (NLTK)": stem,
        "Лемма (spaCy)": lemma_spacy,
        "Лемма (PyMorphy2)": lemma_pymorphy,
        "Лемма (Natasha)": lemma_natasha
    })

# Создаем таблицу для русских слов
comparison_ru_df = pd.DataFrame(comparison_ru)
print("Сравнение методов для русского языка:")
comparison_ru_df

NameError: name 'nlp_ru' is not defined

In [16]:
# Выбираем несколько интересных слов из английского текста для сравнения
english_words = ["States", "people", "diverse", "traditions", "beliefs",
                "Americans", "interested", "cultures", "neighboring", "established", "corresponded", "traveling"]

# Создаем DataFrame для сравнения
comparison_en = []
for word in english_words:
    stem = english_stemmer.stem(word.lower())

    # Находим лемму в NLTK
    pos = pos_tag([word.lower()])[0][1]
    wordnet_pos = get_wordnet_pos(pos)
    lemma_nltk = lemmatizer.lemmatize(word.lower(), wordnet_pos)

    # Находим лемму в spaCy
    doc = nlp_en(word)
    for token in doc:
        lemma_spacy = token.lemma_

    comparison_en.append({
        "Слово": word,
        "Стем (NLTK)": stem,
        "Лемма (NLTK)": lemma_nltk,
        "Лемма (spaCy)": lemma_spacy
    })

# Создаем таблицу для английских слов
comparison_en_df = pd.DataFrame(comparison_en)
print("Сравнение методов для английского языка:")
comparison_en_df

NameError: name 'english_stemmer' is not defined

**2. Комплексные пайплайны препроцессинга текста**

2.1 Пайплайн для русского языка с PyMorphy2

In [8]:
import re
import pymorphy2
from nltk.corpus import stopwords

In [13]:
# Загрузка стоп-слов
nltk.download('stopwords')
russian_stopwords = set(stopwords.words('russian'))

# Инициализация анализатора
morph = pymorphy2.MorphAnalyzer()

def preprocess_russian_text(text):
    """
    Полный пайплайн предобработки русского текста с использованием PyMorphy2
    """
    # Шаг 1: Приведение текста к нижнему регистру
    text = text.lower()
    print(f"1. Приведение к нижнему регистру: {text[:50]}...")

    # Шаг 2: Замена 'ё' на 'е'
    text = text.replace('ё', 'е')
    print(f"2. Замена 'ё' на 'е': {text[:50]}...")

    # Шаг 3: Удаление цифр, знаков препинания и лишних пробелов
    text = re.sub(r'[^\w\s]', ' ', text)  # Удаление знаков препинания
    text = re.sub(r'\d+', ' ', text)      # Удаление цифр
    text = re.sub(r'\s+', ' ', text)      # Удаление лишних пробелов
    text = text.strip()                    # Удаление пробелов в начале и конце
    print(f"3. Очистка текста: {text[:50]}...")

    # Шаг 4: Токенизация
    tokens = text.split()
    print(f"4. Токенизация: получено {len(tokens)} токенов")

    # Шаг 5: Удаление стоп-слов
    filtered_tokens = [token for token in tokens if token not in russian_stopwords]
    print(f"5. Удаление стоп-слов: осталось {len(filtered_tokens)} токенов")

    # Шаг 6: Лемматизация с PyMorphy2
    lemmas = []
    for token in filtered_tokens:
        parsed = morph.parse(token)[0]
        lemmas.append(parsed.normal_form)

    print(f"6. Лемматизация: получено {len(lemmas)} лемм")

    # Шаг 7: Удаление слишком коротких слов
    final_lemmas = [lemma for lemma in lemmas if len(lemma) > 2]
    print(f"7. Фильтрация коротких слов: итоговое количество {len(final_lemmas)} лемм")

    return final_lemmas

# Применение пайплайна к русскому тексту
print("Комплексный пайплайн для русского текста:")
preprocessed_russian = preprocess_russian_text(russian_text)
print("\nРезультат:")
print(preprocessed_russian)

Комплексный пайплайн для русского текста:
1. Приведение к нижнему регистру: 
в россии проживают народы с разными традициями и ...
2. Замена 'ё' на 'е': 
в россии проживают народы с разными традициями и ...
3. Очистка текста: в россии проживают народы с разными традициями и в...
4. Токенизация: получено 39 токенов
5. Удаление стоп-слов: осталось 31 токенов
6. Лемматизация: получено 31 лемм
7. Фильтрация коротких слов: итоговое количество 31 лемм

Результат:
['россия', 'проживать', 'народ', 'разный', 'традиция', 'верование', 'русский', 'человек', 'интерес', 'относиться', 'культура', 'соседний', 'народ', 'пётр', 'первый', 'ввести', 'многие', 'европейский', 'обычай', 'екатерина', 'великий', 'переписываться', 'французский', 'просветитель', 'современный', 'россиянин', 'любить', 'путешествовать', 'разный', 'страна', 'мир']


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


2.2 Пайплайн для английского языка с spaCy

In [ ]:
import spacy
from nltk.corpus import stopwords

In [ ]:
# Загрузка стоп-слов и модели spaCy
english_stopwords = set(stopwords.words('english'))
nlp_en = spacy.load("en_core_web_sm")

def preprocess_english_text(text):
    """
    Полный пайплайн предобработки английского текста с использованием spaCy
    """
    # Шаг 1: Приведение текста к нижнему регистру
    text = text.lower()
    print(f"1. Приведение к нижнему регистру: {text[:50]}...")

    # Шаг 2: Расширение сокращений (контракций)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"'ve", " have", text)
    text = re.sub(r"'re", " are", text)
    text = re.sub(r"'m", " am", text)
    text = re.sub(r"'ll", " will", text)
    text = re.sub(r"'d", " would", text)
    print(f"2. Расширение контракций: {text[:50]}...")

    # Шаг 3: Обработка текста с помощью spaCy
    doc = nlp_en(text)
    print(f"3. Обработка с spaCy: получено {len(doc)} токенов")

    # Шаг 4: Фильтрация и лемматизация
    lemmas = []
    for token in doc:
        # Проверяем, что токен не является стоп-словом, пунктуацией или числом
        if (not token.is_stop and
            not token.is_punct and
            not token.is_digit and
            token.is_alpha):  # Только буквенные токены

            # Получаем лемму и проверяем, что она не является специальным токеном spaCy
            lemma = token.lemma_
            if lemma != '-PRON-':  # spaCy иногда использует -PRON- для местоимений
                lemmas.append(lemma)

    print(f"4. Фильтрация и лемматизация: получено {len(lemmas)} лемм")

    # Шаг 5: Удаление слишком коротких слов
    final_lemmas = [lemma for lemma in lemmas if len(lemma) > 2]
    print(f"5. Фильтрация коротких слов: итоговое количество {len(final_lemmas)} лемм")

    return final_lemmas

# Применение пайплайна к английскому тексту
print("Комплексный пайплайн для английского текста:")
preprocessed_english = preprocess_english_text(english_text)
print("\nРезультат:")
print(preprocessed_english)

2.3 Пайплайн для смешанного текста (русский + английский)

Блок 1: Импорт необходимых библиотек и инициализация

In [ ]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import pymorphy2

# Загрузим необходимые ресурсы
nltk.download('stopwords')
nltk.download('punkt')

# Инициализация анализаторов
morph = pymorphy2.MorphAnalyzer()
russian_stemmer = SnowballStemmer("russian")
english_stemmer = SnowballStemmer("english")

# Списки стоп-слов
russian_stop_words = set(stopwords.words('russian'))
english_stop_words = set(stopwords.words('english'))


In [ ]:
# Исходный текст
mixed_text = """
<p>В современном мире digital-технологии стали неотъемлемой частью жизни.</p>
<strong>Information technologies</strong> изменили способы коммуникации между людьми в 2023 году.
CEO крупных IT-компаний (более 500 сотрудников) регулярно проводят online-конференции.
<a href="https://example.com">Developers</a> из разных стран сотрудничают в open-source проектах.
Искусственный интеллект и <em>machine learning</em> находят применение в разных сферах.
Для связи: contact@tech-info.com или посетите https://tech-conference.org.
<div class="footer">
  © 2023 AI-News. Телефон: +7 (123) 456-78-90. #искусственныйинтеллект #bigdata
</div>
Data-scientists и ML-инженеры работают с web-приложениями и AI-моделями.
Кибер-безопасность и e-commerce — важные направления IT-индустрии.
"""

Блок 2: Нормализация текста

In [ ]:
def normalize_mixed_text(text):
    """
    Нормализует смешанный текст, удаляя HTML-теги, URL, email, специальные символы
    и выполняя базовые преобразования для стандартизации текста.

    Args:
        text (str): Исходный смешанный текст с "шумом"

    Returns:
        str: Нормализованный текст
    """
    # Шаг 1: Удаление HTML-тегов
    # Находит и удаляет всё между < и >, включая скрипты и стили
    text = re.sub(r'<[^>]+>', ' ', text)

    # Шаг 2: Удаление URL-адресов
    text = re.sub(r'https?://\S+|www\.\S+', ' ', text)

    # Шаг 3: Удаление email-адресов
    text = re.sub(r'\S+@\S+\.\S+', ' ', text)

    # Шаг 4: Удаление хештегов
    text = re.sub(r'#\S+', ' ', text)

    # Шаг 5: Удаление номеров телефонов (различные форматы)
    text = re.sub(r'\+\d+\s*[\(\)]*\s*\d+[\s\-\(\)]*\d+[\s\-\(\)]*\d+', ' ', text)

    # Шаг 6: Удаление символов копирайта, специальных символов
    text = re.sub(r'[©®™℠]+', ' ', text)

    # Шаг 7: Замена скобок и их содержимого на пробел
    text = re.sub(r'\([^)]*\)', ' ', text)

    # Шаг 8: Приведение к нижнему регистру
    text = text.lower()

    # Шаг 9: Замена буквы "ё" на "е" для унификации русского текста
    text = text.replace('ё', 'е')

    # Шаг 10: Обработка пунктуации
    # Сохраняем дефисы внутри слов, но удаляем остальную пунктуацию
    # Сначала заменим дефисы временным маркером
    text = re.sub(r'(\w)-(\w)', r'\1HYPHEN\2', text)

    # Удаляем пунктуацию кроме временных маркеров дефисов
    text = re.sub(r'[^\w\sHYPHEN]', ' ', text)

    # Возвращаем дефисы
    text = re.sub(r'HYPHEN', '-', text)

    # Шаг 11: Удаление цифр, но сохранение слов с цифрами (например, ML-инженеры5)
    text = re.sub(r'\b\d+\b', ' ', text)

    # Шаг 12: Удаление избыточных пробелов (в том числе в начале и конце строк)
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# Тест функции
normalized_text = normalize_mixed_text(mixed_text)
print("Нормализованный текст:")
print(normalized_text)

Блок 3: Определение алфавита символа и токенизация

In [ ]:
def is_cyrillic(char):
    """Проверяет, является ли символ кириллическим."""
    return 'а' <= char <= 'я' or char == 'ё'

def is_latin(char):
    """Проверяет, является ли символ латинским."""
    return 'a' <= char <= 'z'

def classify_token(token):
    """Определяет язык токена: 'russian', 'english' или 'mixed'."""
    if not token:
        return None

    # Удаляем все не-буквенные символы для определения языка
    letters = [c for c in token.lower() if c.isalpha()]
    if not letters:
        return None

    cyrillic_count = sum(1 for c in letters if is_cyrillic(c))
    latin_count = sum(1 for c in letters if is_latin(c))

    if cyrillic_count > 0 and latin_count == 0:
        return 'russian'
    elif latin_count > 0 and cyrillic_count == 0:
        return 'english'
    elif cyrillic_count > 0 and latin_count > 0:
        return 'mixed'
    else:
        return None

def split_hyphenated_words(token):
    """Разбивает слова с дефисом, если они состоят из смешанных символов."""
    if '-' not in token:
        return [token]

    # Проверяем, есть ли смешанные символы в токене
    if classify_token(token) != 'mixed':
        return [token]

    # Разбиваем по дефису
    parts = token.split('-')
    result = []

    for part in parts:
        if part:  # Игнорируем пустые части
            result.append(part)

    return result

def tokenize_and_classify(text):
    """Токенизирует текст и классифицирует токены по языку."""
    # Сначала разделим текст на слова, сохраняя дефисы внутри слов
    raw_tokens = re.findall(r'\b[\w\-]+\b', text)

    russian_tokens = []
    english_tokens = []

    for token in raw_tokens:
        # Проверяем, нужно ли разделить слово с дефисом
        split_tokens = split_hyphenated_words(token)

        for t in split_tokens:
            lang = classify_token(t)
            if lang == 'russian':
                russian_tokens.append(t)
            elif lang == 'english':
                english_tokens.append(t)

    return russian_tokens, english_tokens

# Применяем токенизацию и классификацию
russian_tokens, english_tokens = tokenize_and_classify(normalized_text)

print("\nРусские токены:")
print(russian_tokens)
print("\nАнглийские токены:")
print(english_tokens)

Блок 4: Обработка токенов по языковым правилам

In [ ]:
def process_russian_tokens(tokens):
    """Обрабатывает русские токены по пайплайну для русского языка."""
    # Фильтрация стоп-слов
    filtered_tokens = [token for token in tokens if token not in russian_stop_words]

    # Лемматизация с помощью pymorphy2
    lemmatized_tokens = []
    for token in filtered_tokens:
        # Получаем нормальную форму
        lemma = morph.parse(token)[0].normal_form
        lemmatized_tokens.append(lemma)

    return lemmatized_tokens

def process_english_tokens(tokens):
    """Обрабатывает английские токены по пайплайну для английского языка."""
    # Обработка сокращений и апострофов
    expanded_tokens = []
    for token in tokens:
        # Обработка притяжательных форм и сокращений
        token = re.sub(r'\'s$|\'$', '', token)  # Удаляем 's и '
        expanded_tokens.append(token)

    # Фильтрация стоп-слов
    filtered_tokens = [token for token in expanded_tokens if token not in english_stop_words]

    # Стемматизация
    stemmed_tokens = [english_stemmer.stem(token) for token in filtered_tokens]

    return stemmed_tokens

# Обработка токенов по языковым правилам
processed_russian = process_russian_tokens(russian_tokens)
processed_english = process_english_tokens(english_tokens)

print("\nОбработанные русские токены:")
print(processed_russian)
print("\nОбработанные английские токены:")
print(processed_english)

Блок 5: Финальная обработка и объединение результатов

In [ ]:
def final_processing(russian_tokens, english_tokens):
    """Выполняет постобработку и объединяет результаты."""
    # Фильтрация коротких слов (менее 3 символов)
    filtered_russian = [token for token in russian_tokens if len(token) >= 3]
    filtered_english = [token for token in english_tokens if len(token) >= 3]

    # Удаление дубликатов
    unique_russian = list(dict.fromkeys(filtered_russian))
    unique_english = list(dict.fromkeys(filtered_english))

    return {
        'russian_tokens': unique_russian,
        'english_tokens': unique_english,
        'stats': {
            'russian_count': len(unique_russian),
            'english_count': len(unique_english),
            'total_unique': len(unique_russian) + len(unique_english)
        }
    }

# Финальная обработка
result = final_processing(processed_russian, processed_english)

print("\nФинальный результат:")
print("Русские слова:", result['russian_tokens'])
print("Английские слова:", result['english_tokens'])
print("\nСтатистика:")
print(f"Количество русских слов: {result['stats']['russian_count']}")
print(f"Количество английских слов: {result['stats']['english_count']}")
print(f"Общее количество уникальных слов: {result['stats']['total_unique']}")

**Практические упражнения:**

Упражнение 1: Базовая стемматизация русского текста с помощью NLTK

Задание:
Напишите функцию, которая принимает русский текст, выполняет токенизацию и применяет стемматизацию с использованием SnowballStemmer из библиотеки NLTK. Функция должна возвращать список стемматизированных слов.

Требования:

* Используйте SnowballStemmer("russian") из NLTK
* Удалите все знаки препинания перед стемматизацией
* Приведите слова к нижнему регистру
* Выведите исходные слова и их стеммы в виде таблицы для сравнения

In [17]:
from nltk.stem import SnowballStemmer
import re
import pandas as pd

russian_stemmer = SnowballStemmer("russian")

russian_text = """
Программирование - это искусство создания программ для компьютеров.
Программисты разрабатывают различные приложения, которые используются миллионами людей.
Хорошие разработчики всегда учатся новому и совершенствуют свои навыки программирования.
"""

def stem_russian_text(text):
    words = re.findall(r'\b\w+\b', text.lower())

    stemmed_words = [russian_stemmer.stem(word) for word in words]

    comparison = []
    for i, (original, stem) in enumerate(zip(words, stemmed_words)):
        comparison.append({
            '№': i+1,
            'Исходное слово': original,
            'Стемма': stem
        })

    comparison_df = pd.DataFrame(comparison)
    print("Сравнение исходных слов и стемм:")
    print(comparison_df.head(15))

    return comparison_df

result_df = stem_russian_text(russian_text)


Сравнение исходных слов и стемм:
     №    Исходное слово          Стемма
0    1  программирование  программирован
1    2               это              эт
2    3         искусство        искусств
3    4          создания          создан
4    5          программ        программ
5    6               для             для
6    7       компьютеров       компьютер
7    8      программисты     программист
8    9     разрабатывают     разрабатыва
9   10         различные         различн
10  11        приложения        приложен
11  12           которые           котор
12  13      используются         использ
13  14        миллионами         миллион
14  15             людей             люд


Упражнение 2: Сравнение стемматизации и лемматизации английского текста

Задание: Реализуйте две функции, которые обрабатывают один и тот же английский текст с использованием PorterStemmer и WordNetLemmatizer из NLTK соответственно. Сравните результаты и определите, какой метод лучше сохраняет семантику слов.

Требования:

* Для стемматизации используйте PorterStemmer из NLTK
* Для лемматизации используйте WordNetLemmatizer с определением части речи
* Создайте таблицу сравнения из трех колонок: исходное слово, стемма, лемма
* Выделите случаи, где результаты стемматизации и лемматизации существенно различаются

In [21]:
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk import pos_tag, word_tokenize
from nltk.corpus import wordnet
import nltk
import pandas as pd

nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('wordnet')

stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

english_text = """
Natural language processing helps computers understand and generate human language.
Researchers are developing better models that can recognize patterns in texts.
The running applications are processing huge amounts of data daily.
Many universities are teaching students how to build better language models.
"""

def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def compare_stemming_lemmatization(text):
    tokens = word_tokenize(text.lower())
    tagged = pos_tag(tokens)

    comparison = []
    different_count = 0

    for word, tag in tagged:
        if word.isalpha():
            stem = stemmer.stem(word)
            wordnet_pos = get_wordnet_pos(tag)
            lemma = lemmatizer.lemmatize(word, wordnet_pos)

            is_different = stem != lemma

            comparison.append({
                'Слово': word,
                'Стемма': stem,
                'Лемма': lemma,
                'Различаются': 'Да' if is_different else 'Нет'
            })

            if is_different:
                different_count += 1

    comparison_df = pd.DataFrame(comparison)

    print("Сравнение стемматизации и лемматизации:")
    print(comparison_df.head(20))

    print(f"\nКоличество слов с разными стеммами и леммами: {different_count}")

    different_df = comparison_df[comparison_df['Различаются'] == 'Да']
    if len(different_df) > 0:
        print("\nСлова с разными стеммами и леммами:")
        print(different_df.head(10))

    return comparison_df

results_df = compare_stemming_lemmatization(english_text)

Сравнение стемматизации и лемматизации:
          Слово      Стемма       Лемма Различаются
0       natural       natur     natural          Да
1      language     languag    language          Да
2    processing     process  processing          Да
3         helps        help        help         Нет
4     computers      comput    computer          Да
5    understand  understand  understand         Нет
6           and         and         and         Нет
7      generate       gener    generate          Да
8         human       human       human         Нет
9      language     languag    language          Да
10  researchers    research  researcher          Да
11          are         are          be          Да
12   developing     develop     develop         Нет
13       better      better        good          Да
14       models       model       model         Нет
15         that        that        that         Нет
16          can         can         can         Нет
17    recognize      rec

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Упражнение 3: Лемматизация русского текста с использованием pymorphy2 и определение частей речи

Задание: Напишите функцию, которая выполняет морфологический анализ русского текста с помощью библиотеки pymorphy2. Функция должна возвращать для каждого слова его лемму (нормальную форму) и часть речи.

Требования:

* Используйте pymorphy2.MorphAnalyzer()
* Обработайте токенизированный текст, игнорируя пунктуацию и числа
* Для каждого слова определите нормальную форму и часть речи
* Посчитайте частоту встречаемости каждой части речи в тексте
* Найдите слова, имеющие омонимичные разборы, и предложите способ разрешения неоднозначности

In [28]:
import pymorphy2
import re
import pandas as pd

morph = pymorphy2.MorphAnalyzer()

russian_text = """
Машинное обучение становится всё более популярным в современном мире.
Компании используют алгоритмы машинного обучения для анализа данных и принятия решений.
Инженеры по данным обучают модели на больших наборах данных, чтобы повысить их точность.
Эти технологии меняют нашу жизнь, делая её более удобной и эффективной.
"""

def analyze_russian_text(text):
    words = re.findall(r'\b\w+\b', text.lower())

    comparison = []
    pos_dict = {}
    ambiguous_words = []

    for i, word in enumerate(words):
        if word.isalpha():
            parsed = morph.parse(word)[0]
            normal_form = parsed.normal_form
            pos = parsed.tag.POS if parsed.tag.POS else 'UNKN'

            is_ambiguous = len(morph.parse(word)) > 1

            comparison.append({
                '№': i+1,
                'Слово': word,
                'Лемма': normal_form,
                'Часть речи': pos,
                'Омонимия': 'Да' if is_ambiguous else 'Нет'
            })

            if pos in pos_dict:
                pos_dict[pos] += 1
            else:
                pos_dict[pos] = 1

            if is_ambiguous:
                ambiguous_words.append(word)

    comparison_df = pd.DataFrame(comparison)

    print("Лемматизация и части речи:")
    print(comparison_df.head(20))

    print(f"\nСтатистика частей речи:")
    pos_stats = []
    for pos, count in pos_dict.items():
        pos_stats.append({'Часть речи': pos, 'Количество': count})

    def sort_by_count(item):
        return item['Количество']

    pos_stats.sort(key=sort_by_count, reverse=True)
    pos_df = pd.DataFrame(pos_stats)
    print(pos_df)

    if ambiguous_words:
        print(f"\nСлова с омонимией ({len(ambiguous_words)}):")
        ambiguous_data = []
        for i, word in enumerate(ambiguous_words[:10]):
            ambiguous_data.append({'№': i+1, 'Слово': word})

        ambiguous_df = pd.DataFrame(ambiguous_data)
        print(ambiguous_df)

    return comparison_df, pos_df

results_df, pos_stats_df = analyze_russian_text(russian_text)

Лемматизация и части речи:
     №        Слово         Лемма Часть речи Омонимия
0    1     машинное      машинный       ADJF       Да
1    2     обучение      обучение       NOUN       Да
2    3   становится   становиться       VERB      Нет
3    4          всё           всё       PRCL       Да
4    5        более         более       ADVB      Нет
5    6   популярным    популярный       ADJF       Да
6    7            в             в       PREP       Да
7    8  современном   современный       ADJF       Да
8    9         мире           мир       NOUN       Да
9   10     компании      компания       NOUN       Да
10  11   используют  использовать       VERB       Да
11  12    алгоритмы      алгоритм       NOUN       Да
12  13    машинного      машинный       ADJF       Да
13  14     обучения      обучение       NOUN       Да
14  15          для           для       PREP       Да
15  16      анализа        анализ       NOUN      Нет
16  17       данных        данные       NOUN       Да
1

Упражнение 4: Обработка смешанного русско-английского текста с использованием spaCy

Задание: Разработайте систему для обработки смешанного русско-английского текста с использованием библиотеки spaCy. Ваша программа должна определять язык каждого слова, применять соответствующую модель для лемматизации и выполнять частеречную разметку.

Требования:

* Загрузите модели en_core_web_sm и ru_core_news_sm из spaCy
* Реализуйте алгоритм определения языка для каждого токена (можно использовать множества символов кириллицы и латиницы)
* Обработайте каждый токен соответствующей языковой моделью
* Сформируйте два отдельных набора данных: лемматизированный русский и английский текст
* Определите наиболее часто встречающиеся леммы для каждого языка

In [26]:
import spacy
import re
import pandas as pd

nlp_en = spacy.load("en_core_web_sm")
nlp_ru = spacy.load("ru_core_news_sm")

mixed_text = """
Data science становится ключевой компетенцией в современном IT-мире.
Machine learning инженеры и аналитики данных востребованы на рынке труда.
Компании активно внедряют artificial intelligence решения для оптимизации бизнес-процессов.
Работодатели ищут специалистов, владеющих языком программирования Python и имеющих опыт работы с big data.
"""

def process_mixed_text(text):
    def is_cyrillic(char):
        return 'а' <= char <= 'я' or char == 'ё'

    def is_latin(char):
        return 'a' <= char <= 'z'

    def classify_token(token):
        if not token or not token.isalpha():
            return None

        cyrillic_count = 0
        latin_count = 0

        for c in token:
            char_lower = c.lower()
            if is_cyrillic(char_lower):
                cyrillic_count += 1
            elif is_latin(char_lower):
                latin_count += 1

        if cyrillic_count > 0 and latin_count == 0:
            return 'russian'
        elif latin_count > 0 and cyrillic_count == 0:
            return 'english'
        else:
            return 'mixed'

    tokens = re.findall(r'\b\w+\b', text.lower())

    comparison = []
    russian_lemmas = []
    english_lemmas = []

    for i, token in enumerate(tokens):
        lang = classify_token(token)
        lemma = None

        if lang == 'russian':
            doc = nlp_ru(token)
            for tok in doc:
                if tok.is_alpha:
                    lemma = tok.lemma_
                    russian_lemmas.append(lemma)

        elif lang == 'english':
            doc = nlp_en(token)
            for tok in doc:
                if tok.is_alpha:
                    lemma = tok.lemma_
                    english_lemmas.append(lemma)

        comparison.append({
            '№': i+1,
            'Токен': token,
            'Язык': lang or 'mixed',
            'Лемма': lemma or token
        })

    comparison_df = pd.DataFrame(comparison)

    print("Анализ смешанного текста:")
    print(comparison_df.head(20))

    lang_dict = {}
    for item in comparison:
        lang = item['Язык']
        if lang in lang_dict:
            lang_dict[lang] += 1
        else:
            lang_dict[lang] = 1

    lang_stats = []
    for lang, count in lang_dict.items():
        lang_stats.append({'Язык': lang, 'Количество': count})

    lang_stats_df = pd.DataFrame(lang_stats)
    print(f"\nРаспределение по языкам:")
    print(lang_stats_df)

    def get_top_lemmas(lemmas_list, top_n=10):
        lemma_dict = {}
        for lemma in lemmas_list:
            if lemma in lemma_dict:
                lemma_dict[lemma] += 1
            else:
                lemma_dict[lemma] = 1

        lemma_items = list(lemma_dict.items())

        def get_frequency(item):
            return item[1]

        sorted_lemmas = sorted(lemma_items, key=get_frequency, reverse=True)
        return sorted_lemmas[:top_n]

    russian_top = get_top_lemmas(russian_lemmas)
    english_top = get_top_lemmas(english_lemmas)

    russian_freq_df = pd.DataFrame(russian_top, columns=['Лемма', 'Частота'])
    english_freq_df = pd.DataFrame(english_top, columns=['Лемма', 'Частота'])

    print(f"\nТоп-10 русских лемм:")
    print(russian_freq_df)

    print(f"\nТоп-10 английских лемм:")
    print(english_freq_df)

    return comparison_df, lang_stats_df, russian_freq_df, english_freq_df

results_df, lang_stats_df, ru_freq_df, en_freq_df = process_mixed_text(mixed_text)

Анализ смешанного текста:
     №         Токен     Язык         Лемма
0    1          data  english         datum
1    2       science  english       science
2    3    становится  russian   становиться
3    4      ключевой  russian      ключевой
4    5  компетенцией  russian   компетенция
5    6             в  russian             в
6    7   современном  russian   современном
7    8            it  english            it
8    9          мире  russian           мир
9   10       machine  english       machine
10  11      learning  english         learn
11  12      инженеры  russian       инженер
12  13             и  russian             и
13  14     аналитики  russian      аналитик
14  15        данных  russian        данных
15  16  востребованы  russian  востребовать
16  17            на  russian            на
17  18         рынке  russian         рынок
18  19         труда  russian          труд
19  20      компании  russian      компания

Распределение по языкам:
      Язык  Количество
0

Упражнение 5*: Применение библиотеки Natasha для анализа именованных сущностей и лемматизации русского текста

Задание: Используйте библиотеку Natasha для выделения именованных сущностей (NER) и лемматизации русского текста. Создайте функцию, которая принимает текст, выделяет и классифицирует в нём именованные сущности (имена людей, организации, локации и т.д.), а также выполняет лемматизацию всех слов.

Требования:

* Используйте Segmenter, MorphVocab, NewsEmbedding, NewsMorphTagger, NewsSyntaxParser, NewsNERTagger из библиотеки Natasha
* Выделите все именованные сущности в тексте и определите их тип
* Выполните лемматизацию всех слов, учитывая контекст (используя синтаксический анализ)
* Сформируйте структурированный результат, включающий: список всех именованных сущностей с указанием их типа, список всех слов с их леммами, статистику по типам найденных сущностей

In [ ]:
# ваш код

# Текст для проверки

russian_ner_text = """
Президент России Владимир Путин провел совещание с правительством в Москве.
Компания Яндекс представила новую технологию для анализа данных.
Сотрудники Московского государственного университета и Института проблем искусственного интеллекта РАН
разработали алгоритм для обработки естественного языка.
Исследователи из Санкт-Петербурга и их коллеги из Казани опубликовали результаты исследований
в журнале "Компьютерная лингвистика и интеллектуальные технологии".
"""